# Приклад агента для бронювання готелів та авіаквитків

Це рішення допоможе вам забронювати авіаквитки та готель. Сценарій: подорож з Лондона (Heathrow LHR) 20 лютого 2024 року до Нью-Йорка (JFK) з поверненням 27 лютого 2025 року, економ-класом, виключно з British Airways. Я хочу зупинитися в готелі Hilton у Нью-Йорку, будь ласка, надайте вартість авіаквитків та готелю.


# Ініціалізація служби Azure AI Agent та отримання інформації про конфігурацію з **.env**

### **.env**

Створіть файл .env

**.env** містить рядок підключення до служби Azure AI Agent, модель, яку використовує AOAI, а також відповідний API пошукової служби Google, ENDPOINT тощо.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Назва розгортання моделі вашої служби Azure AI Agent"

[**NOTE**] Вам знадобиться модель з лімітом 100,000 токенів на хвилину та лімітом 600 запитів на хвилину.

  Ви можете отримати модель у розділі Azure AI Foundry - Model and Endpoint.

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Рядок підключення до проекту вашої служби Azure AI Agent"

  Ви можете отримати рядок підключення до проекту у розділі огляду вашого проекту на порталі AI Foundry.

- **SERPAPI_SEARCH_API_KEY** = "Ваш ключ API для SERPAPI Search"
- **SERPAPI_SEARCH_ENDPOINT** = "Ваш ENDPOINT для SERPAPI Search"

Щоб отримати назву розгортання моделі та рядок підключення до проекту служби Azure AI Agent, необхідно створити службу Azure AI Agent. Рекомендується використовувати [цей шаблон](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) для прямого створення. (***Примітка:*** Служба Azure AI Agent наразі доступна лише в обмежених регіонах. Рекомендується ознайомитися з [цим посиланням](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) для налаштування регіону.)

Агенту необхідно отримати доступ до SERPAPI. Рекомендується зареєструватися за [цим посиланням](https://serpapi.com/searches). Після реєстрації ви зможете отримати унікальний ключ API та ENDPOINT.


# Налаштування

Щоб запустити цей блокнот, переконайтеся, що ви встановили необхідні бібліотеки, виконавши команду `pip install -r requirements.txt`.


In [ ]:
from semantic_kernel import __version__

__version__

Ваша версія Semantic Kernel повинна бути щонайменше 1.27.2.


Завантажте налаштування та ресурси вашого файлу .env, будь ласка, переконайтеся, що ви додали свої ключі та налаштування і створили локальний файл .env.


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Увійдіть до Azure

Тепер вам потрібно увійти до Azure. Відкрийте термінал і виконайте наступну команду:

```bash
az login
```

Ця команда запропонує вам ввести ваші облікові дані Azure, що дозволить службі Azure AI Agent працювати належним чином.


# Пояснення:
Це змінна, яка зберігає ключ API для доступу до сервісу API SERP (сторінка результатів пошуку). Ключ API — це унікальний ідентифікатор, який використовується для автентифікації запитів, пов’язаних із вашим обліковим записом.

Мета: Мета цього рядка — зберегти ключ API у змінній, щоб його можна було використовувати для автентифікації запитів до сервісу API SERP. Ключ API необхідний для доступу до сервісу та виконання пошукових запитів.
Як отримати ключ API SERP: Щоб отримати ключ API SERP, виконайте наступні загальні кроки на https://serpapi.com (точні кроки можуть відрізнятися залежно від конкретного сервісу API SERP, який ви використовуєте):

Виберіть сервіс API SERP: Існує кілька сервісів API SERP, таких як SerpAPI, Google Custom Search JSON API та інші. Виберіть той, який найкраще відповідає вашим потребам.

Зареєструйте обліковий запис: Перейдіть на вебсайт обраного сервісу API SERP і зареєструйте обліковий запис. Можливо, вам доведеться надати базову інформацію та підтвердити свою електронну адресу.

Створіть ключ API: Після реєстрації увійдіть до свого облікового запису та перейдіть до розділу API або панелі керування. Знайдіть опцію для створення або генерації нового ключа API.
Скопіюйте ключ API у ваш .env файл.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# Пояснення:
BASE_URL: Це змінна, яка зберігає базову URL-адресу для кінцевої точки API SERP. Ім'я змінної BASE_URL є загальноприйнятим для позначення того, що ця URL-адреса є відправною точкою для виконання запитів до API.  
'https://serpapi.com/search':  

Це фактичний рядок URL-адреси, призначений змінній BASE_URL. Він представляє кінцеву точку для виконання пошукових запитів за допомогою API SERP.

# Мета:
Мета цього рядка — визначити константу, яка містить базову URL-адресу для API SERP. Ця URL-адреса буде використовуватися як відправна точка для створення запитів до API для виконання пошукових операцій.

# Використання:
Визначивши базову URL-адресу у змінній, ви можете легко використовувати її повторно у своєму коді щоразу, коли потрібно виконати запити до API SERP. Це робить ваш код більш підтримуваним і зменшує ризик помилок, пов'язаних із жорстким кодуванням URL-адреси в кількох місцях. Поточний приклад — https://serpapi.com/search?engine=bing, який використовує API пошуку Bing. Інший API можна вибрати на https://Serpapi.com


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# Пояснення:

Тут знаходиться код вашого плагіна.

Визначення класу: `class BookingPlugin`: Визначає клас під назвою BookingPlugin, який містить методи для бронювання готелів та авіарейсів.

Метод бронювання готелів:

- `@kernel_function(description="booking hotel")`: Декоратор, який описує функцію як ядрову функцію для бронювання готелів.
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`: Визначає метод для бронювання готелів з анотованими параметрами та типом повернення.

Метод створює словник параметрів для запиту бронювання готелю та надсилає GET-запит до SERP API. Він перевіряє статус відповіді та повертає властивості готелю, якщо запит успішний, або None, якщо запит не вдався.

Метод бронювання авіарейсів:

- `@kernel_function(description="booking flight")`: Декоратор, який описує функцію як ядрову функцію для бронювання авіарейсів.
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`: Визначає метод для бронювання авіарейсів з анотованими параметрами та типом повернення.

Метод створює словники параметрів для запитів на рейс "туди" та "назад" і надсилає GET-запити до SERP API. Він перевіряє статус відповіді та додає інформацію про рейс до рядка результату, якщо запит успішний, або виводить повідомлення про помилку, якщо запит не вдався. Метод повертає рядок результату, що містить інформацію про рейс.


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# Пояснення:
Імпорт модулів: Імпортуйте необхідні модулі для роботи з обліковими даними Azure, AI-агентом, вмістом повідомлень чату, роллю автора та декоратором функцій ядра.

Асинхронний контекстний менеджер: `async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,)`: Налаштовує асинхронний контекстний менеджер для роботи з обліковими даними Azure і створення клієнта AI-агента.

Ім'я агента та інструкції:
- `AGENT_NAME = "BookingAgent"`: Визначає ім'я агента.
- `AGENT_INSTRUCTIONS = """..."""`: Надає детальні інструкції для агента щодо обробки запитів на бронювання.

Створення визначення агента: `agent_definition = await client.agents.create_agent(...)`: Створює визначення агента із зазначеною моделлю, ім'ям та інструкціями.

Створення AzureAI-агента: `agent = AzureAIAgent(...)`: Створює AzureAI-агента, використовуючи клієнта, визначення агента та визначений плагін.

Створення потоку: `thread: AzureAIAgentThread | None = None`: Створює потік для агента. Попереднє створення потоку не є обов’язковим — якщо вказано значення `None`, новий потік буде створено під час першого виклику і повернуто як частину відповіді.

Вхідні дані користувача: `user_inputs = ["..."]`: Визначає список вхідних даних користувача, які агент має обробити.

У блоці `finally` видаліть потік і агента, щоб звільнити ресурси.


# Аутентифікація

Клас `DefaultAzureCredential` є частиною Azure SDK для Python. Він забезпечує стандартний спосіб аутентифікації з сервісами Azure. Клас намагається виконати аутентифікацію за допомогою кількох методів у певному порядку, таких як змінні середовища, керована ідентичність і облікові дані Azure CLI.

Асинхронні операції: Модуль aio вказує на те, що клас DefaultAzureCredential підтримує асинхронні операції. Це означає, що ви можете використовувати його з asyncio для виконання запитів на аутентифікацію без блокування.


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)


---

**Відмова від відповідальності**:  
Цей документ був перекладений за допомогою сервісу автоматичного перекладу [Co-op Translator](https://github.com/Azure/co-op-translator). Хоча ми прагнемо до точності, будь ласка, майте на увазі, що автоматичні переклади можуть містити помилки або неточності. Оригінальний документ на його рідній мові слід вважати авторитетним джерелом. Для критичної інформації рекомендується професійний людський переклад. Ми не несемо відповідальності за будь-які непорозуміння або неправильні тлумачення, що виникають внаслідок використання цього перекладу.
